In [ ]:
%qtconsole

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy.fftpack import fft, fftfreq
from parametricaudio import ParametricAudioModel

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
output_notebook()

dB = lambda x: 20*np.log10(np.abs(x))

## Model 
Creates a model for the parametric audio system.
To avoid the kinks from applying the modulation, the carrier is padded a number of cycles at each end, without modulation.
The input signal is smoothed at both ends to avoid the sharp transision between modulation and no modulation.

In [ ]:
fs=192e3 # Default for 40kHz carrier
#fs=450e3
model = ParametricAudioModel(
    modulation='sqrt',demodulation='envelope',
    fs=fs,padcycles=100,windowcycles=400,
    lpf=True,equalize='frequency')

## Two sines
An input signal consisting of two sine waves, without noise.

In [ ]:
periods = 200.33
f0 = 1.13e3
f0=1e3
#f0=200
n = np.ceil(periods*fs/f0).astype('int')
n=2**15
t=np.arange(n)/fs

inputsignal = np.sin(2*np.pi*f0*t) + 0.5*np.sin(4.5*np.pi*f0*t)
inputsignal = inputsignal/np.max(np.abs(inputsignal))
outputsignal = model(inputsignal)

In [ ]:
f1 = figure()
f1.line(t,inputsignal,legend='Input')
#f1.line(model.t,model.signal,legend='Equalized signal',color='orange')
f1.line(t,outputsignal,legend='Output',color='Green')
#ax[0].plot(inputsignal[idx],label='Input',linewidth=4)
#ax[0].plot(outputsignal[idx],label='Output')
#ax[0].plot(model.window[idx],label='Window')
#ax[0].legend(loc='lower right')
#ax[0].set_title('Time signals')

f2 = figure(x_axis_type='log')
f2.line(fftfreq(inputsignal.size,1/fs),dB(fft(inputsignal)),legend='Input')
#f2.line(fftfreq(model.signal.size,1/fs),dB(fft(model.signal)),legend='Equalized input',color='orange')
f2.line(fftfreq(outputsignal.size,1/fs),dB(fft(outputsignal)),legend='Output',color='green')
p = gridplot([[f1],[f2]])
show(p)
#ax[1].semilogx(f[idx],dB(inputspectrum[idx]),label='Input',linewidth=4)
#ax[1].semilogx(f[idx],dB(outputspectrum[idx]),label='Output')
#ax[1].legend()
#ax[1].set_title('Frequency spectra')
#mpld3.display()
#plt.show()

## Noisy sine signal

In [ ]:
periods = 100
f0 = 1.13e3
n = np.ceil(periods*fs/f0).astype('int')
t=np.arange(n)/fs

inputsignal = np.sin(2*np.pi*f0*t) +1*(2*np.random.rand(n)-1)
inputsignal = (2*np.random.rand(n)-1)
lpf = signal.iirfilter(6,24e3/(fs/2),btype='low')
inputsignal = signal.lfilter(lpf[0], lpf[1], inputsignal)
inputsignal = inputsignal/np.max(np.abs(inputsignal))
outputsignal = model(inputsignal)

In [ ]:
idx = np.arange(np.floor(10/periods*n).astype('int'))
fig,ax = plt.subplots(2,1,figsize=(12,12))
ax[0].plot(inputsignal[idx],label='Input',linewidth=4)
ax[0].plot(outputsignal[idx],label='Output')
ax[0].plot(model.window[idx],label='Window')
ax[0].legend(loc='lower right')
ax[0].set_title('Time signals')

inputspectrum = fft(inputsignal)/n
outputspectrum = fft(outputsignal)/n
f = fftfreq(n,1/fs)
idx=(f<20e3) & (f>100)

ax[1].semilogx(f[idx],dB(inputspectrum[idx]),label='Input',linewidth=4)
ax[1].semilogx(f[idx],dB(outputspectrum[idx]),label='Output')
ax[1].legend()
ax[1].set_title('Frequency spectra')
mpld3.display()

## Pure noise signal

In [ ]:
periods = 100
f0 = 1.13e3
n = np.ceil(periods*fs/f0).astype('int')
t=np.arange(n)/fs

inputsignal = 1*(2*np.random.rand(n)-1)
lpf = signal.iirfilter(6,24e3/(fs/2),btype='low')
inputsignal = signal.lfilter(lpf[0], lpf[1], inputsignal)
inputsignal = inputsignal/np.max(np.abs(inputsignal))
outputsignal = model(inputsignal)

In [ ]:
idx = np.arange(np.floor(10/periods*n).astype('int'))
fig,ax = plt.subplots(2,1,figsize=(12,12))
ax[0].plot(inputsignal[idx],label='Input',linewidth=4)
ax[0].plot(outputsignal[idx],label='Output')
ax[0].plot(model.window[idx],label='Window')
ax[0].legend(loc='lower right')
ax[0].set_title('Time signals')

inputspectrum = fft(inputsignal)/n
outputspectrum = fft(outputsignal)/n
f = fftfreq(n,1/fs)
idx=(f<20e3) & (f>100)

ax[1].semilogx(f[idx],dB(inputspectrum[idx]),label='Input',linewidth=4)
ax[1].semilogx(f[idx],dB(outputspectrum[idx]),label='Output')
ax[1].legend()
ax[1].set_title('Frequency spectra')
mpld3.display()